In [9]:
import imageio

dds = 'DDS/sample_1920×1280.dds'
image  = imageio.imread(dds,format='dds')
imageio.imwrite('PNG/image.png',image,format='png')

C:\Users\hp\AppData\Local\Temp\ipykernel_16120\430891327.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image  = imageio.imread(dds,format='dds')


In [2]:
import vpk
import os

vpk_path = 'VPK/tf2_misc_dir.vpk'
output_dir = 'ExtractedVPK/'

with vpk.open(vpk_path) as archive:
    for file_path in archive:
        if file_path.endswith('.vtf'):
            print("Extracting:", file_path)
            file_data = archive.read_file(file_path)
            
            file_name = os.path.basename(file_path)
            output_path = os.path.join(output_dir, file_name)
            with open(output_path, 'wb') as f:
                f.write(file_data)

print("✅ Extracted all .vtf files.")

✅ Extracted all .vtf files.


In [8]:
import os
import subprocess

vtf_folder = r"ExtractedVPK\\"
png_output = r"PNG"
vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe" 

# Make sure the output folder exists
os.makedirs(png_output, exist_ok=True)

# Loop through and convert
for filename in os.listdir(vtf_folder):
    if filename.lower().endswith(".vtf"):
        input_path = os.path.join(vtf_folder, filename)
        print("🔄 Converting:", input_path)
        subprocess.run([
            vtfcmd_path,
            "-file", input_path,
            "-output", png_output,
            "-exportformat", "png"
        ])


🔄 Converting: ExtractedVPK\\donev2.vtf


In [18]:
pip install vtf2img

  Obtaining dependency information for vtf2img from https://files.pythonhosted.org/packages/78/04/8248534c3b357e0aaceae5b7728ff49e2244bfd8858962cc53ad77716433/vtf2img-0.1.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import subprocess
import re
import json

vtfcmd_path = r"vtflib132-bin/bin/x64/VTFCmd.exe"
vtf_file = r"ExtractedVPK\\sky114.vtf"
json_output = r"Metadata/donev2_metadata.json"


result = subprocess.run(
    [vtfcmd_path, "-file", vtf_file , "-output", png_output ,"-exportformat","png"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True  # gets output as string instead of bytes
)

output = result.stdout
print(output)  # you can parse this using regex or string splitting

metadata = {}

# Simple regex matches
patterns = {
    "version": r"Version:\s*v(\d+\.\d+)",
    "width": r"Width:\s*(\d+)",
    "height": r"Height:\s*(\d+)",
    "depth": r"Depth:\s*(\d+)",
    "frames": r"Frames:\s*(\d+)",
    "start_frame": r"Start Frame:\s*(\d+)",
    "faces": r"Faces:\s*(\d+)",
    "mipmaps": r"Mipmaps:\s*(\d+)",
    "flags": r"Flags:\s*(0x[0-9A-Fa-f]+)",
    "format": r"Format:\s*([^\n\r]+)",
    "reflectivity": r"Reflectivity:\s*([\d\.]+),\s*([\d\.]+),\s*([\d\.]+)"
}

for key, pattern in patterns.items():
    match = re.search(pattern, output)
    if match:
        if key == "reflectivity":
            metadata[key] = {
                "r": float(match.group(1)),
                "g": float(match.group(2)),
                "b": float(match.group(3))
            }
        elif key in ["width", "height", "depth", "frames", "start_frame", "faces", "mipmaps"]:
            metadata[key] = int(match.group(1))
        elif key == "version":
            metadata[key] = float(match.group(1))
        else:
            metadata[key] = match.group(1).strip()

# Save metadata to JSON
import os
os.makedirs(os.path.dirname(json_output), exist_ok=True)
with open(json_output, "w") as f:
    json.dump(metadata, f, indent=4)

print("✅ Metadata saved to", json_output)


Processing ExtractedVPK\\sky114.vtf...
 Information:
  Version: v7.5
  Size On Disk: 4096.30 KB
  Width: 1024
  Height: 1024
  Depth: 1
  Frames: 1
  Start Frame: 65535
  Faces: 6
  Mipmaps: 11
  Flags: 0x00004000
  Bumpmap Scale: 1.00
  Reflectivity: 0.07, 0.01, 0.01
  Format: DXT1

  Resources: 2
 Creating texture:
  Writing PNG\sky114.png...
  PNG\sky114.png written.

ExtractedVPK\\sky114.vtf processed.

1/1 files completed.

✅ Metadata saved to Metadata/donev2_metadata.json


In [21]:
from vtf2img import Parser


vtf_file = "ExtractedVPK/sky114.vtf"

parser = Parser(vtf_file)
header = parser.header

print(header.height,header.width)


image = parser.get_image()

image.show()
image.save("donev2.png")

1024 1024
